<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [3]:
d_tweets=d_tweets_dataset['tweet']

In [4]:
non_d_tweets=non_d_tweets_dataset['tweet']

In [5]:
y1=np.ones((3082,1),dtype="int")

In [6]:
y2=np.zeros((2505,1),dtype="int")

In [7]:
y=np.concatenate((y1,y2),axis=0)

In [8]:
res=pd.DataFrame(y)

In [9]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [10]:
txt.shape
txt=txt.reset_index(drop="true")

In [11]:
df=pd.concat([txt,res],axis=1,join="inner")

In [12]:
df.columns=["text","result"]


In [13]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

In [15]:
from gensim.models import FastText

# Train FastText model
fasttext_model = FastText(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [16]:
# Get the FastText word vectors
word_vectors = fasttext_model.wv


In [17]:
# Convert text to FastText word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100)
# X_fasttext_padded
# y = df['result'].values


In [18]:
# Prepare target variable y (same as before)

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [19]:
X_fasttext = []
for sentence in tokenized_text:
    embedding = []
    for word in sentence:
        if word in word_vectors:
            embedding.append(word_vectors[word])
    X_fasttext.append(embedding)

# Find the maximum sequence length in X_fasttext
max_sequence_length = max(len(embedding) for embedding in X_fasttext)

# Pad sequences to the maximum length
X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100, dtype='float32', padding='post')
X_fasttext_padded.shape
y = df['result'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext_padded, y, test_size=0.2, random_state=42)

In [24]:
# Define the LSTM-RNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(100, 100)))

# Add more layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer for regularization

# # Add another LSTM layer with 64 units, dropout, and recurrent dropout
# model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))

# # Add more layers
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))  # Dropout layer for regularization

# Output layer with a single unit and sigmoid activation function for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Step 7: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 8: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')
# Step 9: Make Predictions (same as before)


Epoch 1/10
 46/140 [========>.....................] - ETA: 24s - loss: 0.6898 - accuracy: 0.5564

KeyboardInterrupt: ignored

Using Glove

In [25]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


OSError: ignored

In [ ]:
!pip install spacy

In [26]:
!python -m spacy download en_core_web_md

2023-11-04 06:47:04.173718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 06:47:04.173792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 06:47:04.173838: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 06:47:06.355076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [27]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


In [28]:
# Convert text to GloVe word embeddings
X_glove = []
for sentence in tokenized_text:
    embedding = [get_word_vector(word) for word in sentence]
    embedding = [word_embedding for word_embedding in embedding if word_embedding is not None]
    X_glove.append(embedding)

# Find the maximum sequence length in X_glove
max_sequence_length = max(len(embedding) for embedding in X_glove)

# Pad sequences to the maximum length
X_glove_padded = pad_sequences(X_glove, maxlen=max_sequence_length, dtype='float32', padding='post')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_glove_padded, y, test_size=0.2, random_state=42)


In [31]:
# Define the LSTM-RNN Model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_sequence_length, 300)))  # 300 is the dimensionality of GloVe vectors
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# # Step 6: Make Predictions
# predictions = model.predict_classes(X_test)


Epoch 1/10
140/140 [==============================] - 17s 95ms/step - loss: 0.6367 - accuracy: 0.6386 - val_loss: 0.5553 - val_accuracy: 0.7138
Epoch 2/10
140/140 [==============================] - 13s 93ms/step - loss: 0.5817 - accuracy: 0.6896 - val_loss: 0.5531 - val_accuracy: 0.7138
Epoch 3/10
140/140 [==============================] - 13s 95ms/step - loss: 0.5774 - accuracy: 0.6932 - val_loss: 0.5355 - val_accuracy: 0.7236
Epoch 4/10
140/140 [==============================] - 13s 93ms/step - loss: 0.5634 - accuracy: 0.7024 - val_loss: 0.5388 - val_accuracy: 0.7299
Epoch 5/10
140/140 [==============================] - 13s 93ms/step - loss: 0.5688 - accuracy: 0.6961 - val_loss: 0.5296 - val_accuracy: 0.7254
Epoch 6/10
140/140 [==============================] - 13s 93ms/step - loss: 0.5487 - accuracy: 0.7069 - val_loss: 0.5376 - val_accuracy: 0.6977
Epoch 7/10
140/140 [==============================] - 13s 94ms/step - loss: 0.5521 - accuracy: 0.7089 - val_loss: 0.5366 - val_accuracy:

Using TF-IDF Vectorization

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [33]:
X = df['text']
y = df['result']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [37]:
vocab_size = len(tokenizer.word_index) + 1
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [43]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

In [44]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [45]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
126/126 [==============================] - 21s 149ms/step - loss: 0.6807 - accuracy: 0.5848 - val_loss: 0.6746 - val_accuracy: 0.6667
Epoch 2/10
126/126 [==============================] - 17s 135ms/step - loss: 0.6011 - accuracy: 0.6144 - val_loss: 0.6222 - val_accuracy: 0.6219
Epoch 3/10
126/126 [==============================] - 17s 133ms/step - loss: 0.5858 - accuracy: 0.6638 - val_loss: 0.6308 - val_accuracy: 0.6600
Epoch 4/10
126/126 [==============================] - 17s 137ms/step - loss: 0.5804 - accuracy: 0.6939 - val_loss: 0.6203 - val_accuracy: 0.6644
Epoch 5/10
126/126 [==============================] - 17s 136ms/step - loss: 0.5674 - accuracy: 0.7106 - val_loss: 0.6159 - val_accuracy: 0.6868
Epoch 6/10
126/126 [==============================] - 17s 135ms/step - loss: 0.5566 - accuracy: 0.7245 - val_loss: 0.6155 - val_accuracy: 0.6846
Epoch 7/10
126/126 [==============================] - 18s 142ms/step - loss: 0.5432 - accuracy: 0.7392 - val_loss: 0.5972 - val_ac

In [46]:
y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)  # Apply thresholding at 0.5 for binary classification

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

35/35 [==============================] - 2s 34ms/step
Accuracy: 69.50%
